In [1]:
import mesa
import numpy as np

In [2]:
class Hand:
    def __init__(self, model):
        self.model = model
        self.quartet_groups = [QuartetGroup(quartet_set.group_name, quartet_set.card_names) 
                      for quartet_set in self.model.quartet_sets]
    
    def getCards(self):
        return [quarted_group.getCards() for quarted_group in self.quartet_groups]
    
    def addCard(self, new_card):
        new_card_group_name = new_card.group_name
        for quartet_group in self.quartet_groups:
            if quartet_group.group_name == new_card_group_name:
                quartet_group.addCard(new_card)
    
    def missingCards(self):
        return [group.missingCards() for group in self.quartet_groups]

class QuartetGroup:
    def __init__(self, group_name, card_names):
        self.group_name = group_name
        self.card_names = card_names
        self.cards = [QuartetCard(group_name, card_name, self) for card_name in card_names]
    
    def addCard(self, new_card):
        for card in self.cards:
            if new_card == card:
                card.owned = True
    
    def missingCards(self):
        current_cards_names = self.getCards()
        return [x for x in self.card_names if x not in current_cards_names]
    
    def getCards(self):
        return [x.card_name for x in self.cards if x.owned]
    
    def __repr__(self):
        return str([card for card in self.cards])

class QuartetCard:
    def __init__(self, group_name, card_name, quartet_group, owned = False):
        self.group_name = group_name
        self.card_name = card_name
        self.quartet_group = quartet_group
        self.owned = owned
    
    def __eq__(self, other): 
        if not isinstance(other, QuartetCard):
            return NotImplemented

        return self.group_name == other.group_name and self.card_name == other.card_name
    
    def __repr__(self):
        return self.group_name + "_" + self.card_name
    
    def __hash__(self):
        return hash((self.group_name, self.card_name))

class QuartetAgent(mesa.Agent):
    
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.hand = Hand(model)
        
    def step(self):
        print("I am agent", self.unique_id)
        print("I currently have these cards:")
        print(self.hand.getCards())
        print("I currently do not have these cards:")
        print(self.hand.missingCards())
        other_agent = self.random.choice(self.model.schedule.agents)
    
    def addCard(self, new_card):
        self.hand.addCard(new_card)
    

class QuartetModel(mesa.Model):
    
    def __init__(self, N):
        self.num_agents = N
        self.schedule = mesa.time.BaseScheduler(self)
        self.quartet_sets = [QuartetGroup('a', ['a_1', 'a_2', 'a_3', 'a_4']), 
                             QuartetGroup('b', ['b_1', 'b_2', 'b_3', 'b_4']), 
                             QuartetGroup('c', ['c_1', 'c_2', 'c_3', 'c_4'])]
        # Create agents
        for i in range(self.num_agents):
            a = QuartetAgent(i, self)
            self.schedule.add(a)
        
        all_cards = []
        for quartet_set in self.quartet_sets:
            for quartet_card in quartet_set.cards:
                all_cards.append(quartet_card)
        np.random.shuffle(all_cards)
        
        cards_per_agent = 4
        for i in range(cards_per_agent):
            for agent in self.schedule.agents:
                current_card = all_cards.pop()
                agent.addCard(current_card)
                

    def step(self):
        """Advance the model by one step."""
        self.schedule.step()

In [3]:
model = QuartetModel(3)

In [4]:
model.step()

I am agent 0
I currently have these cards:
[['a_2'], ['b_3'], ['c_1', 'c_2']]
I currently do not have these cards:
[['a_1', 'a_3', 'a_4'], ['b_1', 'b_2', 'b_4'], ['c_3', 'c_4']]
I am agent 1
I currently have these cards:
[['a_4'], ['b_4'], ['c_3', 'c_4']]
I currently do not have these cards:
[['a_1', 'a_2', 'a_3'], ['b_1', 'b_2', 'b_3'], ['c_1', 'c_2']]
I am agent 2
I currently have these cards:
[['a_1', 'a_3'], ['b_1', 'b_2'], []]
I currently do not have these cards:
[['a_2', 'a_4'], ['b_3', 'b_4'], ['c_1', 'c_2', 'c_3', 'c_4']]
